# HA5

Task1

In [311]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [312]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
s = Service('../chromedriver.exe')

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np
import pandas as pd
import time

In [388]:
# Логинимся на сайте

driver = webdriver.Chrome(service = s)
driver.maximize_window()
driver.get('https://mail.ru')

login = driver.find_element(By.NAME, 'login')
login.send_keys('study.ai_172@mail.ru')
login.send_keys(Keys.ENTER)

time.sleep(3)

pwd = driver.find_element(By.NAME, 'password')
pwd.send_keys('NewPassword172#')
pwd.send_keys(Keys.ENTER)

time.sleep(10)
driver.find_element(By.XPATH, '//a[@href = "/inbox/"]').click()
time.sleep(3)
driver.refresh()


"""
# не совсем понял почему через EC не сработало
wait = WebDriverWait(driver, 10000)
element = driver.find_element(By.XPATH, '//a[@href = "/inbox/"]')
item_wait = wait.until(EC.element_to_be_selected((driver.find_element(By.XPATH, '//a[@href = "/inbox/"]'))))
item_wait.click()
"""

'\nwait = WebDriverWait(driver, 10000)\nelement = driver.find_element(By.XPATH, \'//a[@href = "/inbox/"]\')\nitem_wait = wait.until(EC.element_to_be_selected((driver.find_element(By.XPATH, \'//a[@href = "/inbox/"]\'))))\nitem_wait.click()\n'

In [389]:
# Забор ссылок на письма
time.sleep(3)

actions = ActionChains(driver)

links = []

while True:
    mails = driver.find_elements(By.XPATH, 
            '//a[contains(@class, "js-tooltip-direction_letter-bottom") and contains(@class, "js-letter-list-item")]')

    new_links = [mail.get_attribute('href') for mail in mails]
    
    
    if links:
        if links[-1] == new_links[-1]:
            print('Дошли до конца')
            break
    
    links.extend([mail.get_attribute('href') for mail in mails])    
    #actions.move_to_element(mails[-1]).perform()
    mails[-1].location_once_scrolled_into_view
    time.sleep(3)

links = list(set(links))
print('Количество писем = ', len(links))

Дошли до конца
Количество писем =  783


In [394]:
# Забор данных о письмах

titles = []
senders = []
dates = []
bodies = []

#только первые три сделаем для примера, все будет долго думаю
for item in links[:3]:

    driver.get(item)
    time.sleep(3)
    
    title = driver.find_element(By.XPATH, '//h2[contains(@class, "thread__subject")]').text
    titles.append(title)
    
    sender = driver.find_element(By.XPATH, '//span[contains(@class, "letter-contact")]').text
    senders.append(sender) 
    
    datetime = driver.find_element(By.XPATH, '//div[contains(@class, "letter__date")]').text
    dates.append(datetime)   
    
    body = driver.find_element(By.XPATH, '//div[contains(@class, "letter__body")]').text
    bodies.append(body)

# просто проверяем что количество собранных писем совпадает с количеством собранных данных о письмах
len(titles) == len(links)

# не стал уже переводить это в БД, понятно что это можно легко сделать